In [514]:
import os
os.environ["PGPASSWORD"] = "p4ssword"

#Entorno usado en el test
#Postgre 10
#Python 3.8
#Mongo 5.6

# Cambia los datos de conexion (-h, -p) si tu servidor postgres está en otro equipo
!pg_dump -h localhost -p 5432 -sx -U warpcom -d warpcom

--
-- PostgreSQL database dump
--

-- Dumped from database version 10.16
-- Dumped by pg_dump version 10.16

SET statement_timeout = 0;
SET lock_timeout = 0;
SET idle_in_transaction_session_timeout = 0;
SET client_encoding = 'UTF8';
SET standard_conforming_strings = on;
SELECT pg_catalog.set_config('search_path', '', false);
SET check_function_bodies = false;
SET xmloption = content;
SET client_min_messages = warning;
SET row_security = off;

--
-- Name: topology; Type: SCHEMA; Schema: -; Owner: warpcom
--

CREATE SCHEMA topology;


ALTER SCHEMA topology OWNER TO warpcom;

--
-- Name: SCHEMA topology; Type: COMMENT; Schema: -; Owner: warpcom
--

COMMENT ON SCHEMA topology IS 'PostGIS Topology schema';


--
-- Name: plpgsql; Type: EXTENSION; Schema: -; Owner: 
--

CREATE EXTENSION IF NOT EXISTS plpgsql WITH SCHEMA pg_catalog;


--
-- Name: EXTENSION plpgsql; Type: COMMENT; Schema: -; Owner: 
--

COMMENT ON EXTENSION plpgsql IS 'PL/pgSQL procedural language';


--
-- Name: postgis; Type:

In [515]:
import os
os.environ["PGPASSWORD"] = "p4ssword"

#Versión de postgis: 3.2.1 X64

# Cambia los datos de conexion (-h, -p) si tu servidor postgres está en otro equipo
!psql -h localhost -p 5432 -U warpcom -d warpcom -c "\dx"

                                     List of installed extensions
       Name       | Version |   Schema   |                        Description                         
------------------+---------+------------+------------------------------------------------------------
 plpgsql          | 1.0     | pg_catalog | PL/pgSQL procedural language
 postgis          | 3.2.1   | public     | PostGIS geometry and geography spatial types and functions
 postgis_topology | 3.2.1   | topology   | PostGIS topology spatial types and functions
(3 rows)



In [516]:
import os
os.environ["PGPASSWORD"] = "p4ssword"

'''
Por favor, leer la lineas 90 y 91 del archivo ddls.sql para tener en cuenta
un cambio que consideré necesario que hice en la BD.
'''

# Cambia los datos de conexion (-h, -p) si tu servidor postgres está en otro equipo
!psql -h localhost -p 5432 -U warpcom -d warpcom -f model/ddls.sql
!psql -h localhost -p 5432 -U warpcom -d warpcom -c "\d environment_weatherobserved"

DROP TABLE
CREATE TABLE
CREATE INDEX
CREATE INDEX
                           Table "public.environment_weatherobserved"
             Column              |           Type           | Collation | Nullable |  Default   
---------------------------------+--------------------------+-----------+----------+------------
 timeinstant                     | timestamp with time zone |           | not null | 
 location                        | geometry                 |           |          | 
 address                         | text                     |           |          | 
 source                          | text                     |           |          | 
 dataprovider                    | text                     |           |          | 
 name                            | text                     |           |          | 
 battery                         | double precision         |           |          | 
 weathertype                     | text                     |           |          |

In [517]:
# Puedes cambiar el host y puerto si tu servidor mongo es accesible en otra dirección
# Puedes añadir los parámetros -u <username> y -p <password> si tu servidor mongo usa autenticación

'''
Como importar el archivo JSON en Mongodb:
1- Definir la base de datos.
2- Definir la colección
3- Usar uno de los métodos de insernción de datos que provee el shell 
(ej. db.entities.insertOne())
'''

!mongosh localhost:27017/warpcom --eval "db.entities.find({'id': 'ENT-WO-001'})"

Current Mongosh Log ID:	62a58f26fee70a78e8d4ccc8
Connecting to:		mongodb://localhost:27017/warpcom?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+1.5.0
Using MongoDB:		5.0.9
Using Mongosh:		1.5.0

For mongosh info see: https://docs.mongodb.com/mongodb-shell/

------
   The server generated these startup warnings when booting
   2022-06-11T04:53:16.097-07:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted
------

------
   Enable MongoDB's free cloud-based monitoring service, which will then receive and display
   metrics about your deployment (disk utilization, CPU, operation statistics, etc).
   
   The monitoring data will be available on a MongoDB website with a unique URL accessible to you
   and anyone you share the URL with. MongoDB may use this information to make product
   improvements and to suggest MongoDB products and deployment options to you.
   
   To enable free monitoring, run the fol

In [518]:
import csv
import json
from typing import List, Any

'''

https://bugs.python.org/issue42583

Hago referencia a este articulo puesto que a pesar de que el resultado del
código da como se espera, este resultado se ve correcto en la consola mas
no en el archivo es si, es decir, si se abre el archivo JSON que se ha
guardado en este proyecto se podra observar que las comillas simples se
han convertido en comillas dobles, esto es por defecto en la función
json.dumps

Se trata el documento CSV de tal forma que se aprovecha el nombre de las
columnas directamente para extraer el entityid y el entitytype automáticamente,
de esta manera la inserción de la data se puede hacer directamente porque ya
se tienen estos nombres de columnas asociados al JSON.

'''
jsonFilePath = r'model/entities.json'

def load_csv(path: str) -> List[Any]:
    # TODO: implementa esta función
    print("Importar un documento csv y convertirlo en JSON")
    data = construir_json(path, jsonFilePath)
    return data

def construir_json(csvFilePath, jsonFilePath):
    
    ps = ['<','[','(','{','>',']',')','}'] #possible_separators
    data = list()
    #json_obj = {}
    
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)        
        
        for rows in csvReader:       
            
            items = list(rows.items())
            json_obj = {}

            for item in items:
                #print(item)
                #print("Dato: ",item[0])
                #print("Valor: ",item[1])

                #Una vez procesada la data del csv en una lista se arma el JSON
                ps_count = 0
                ps_found = False
                indice_a = 0
                type = ''
                for x in ps:
                    if x in item[0]:
                        #print("True")
                        #Se debe encontrar el tipo de dato
                        indice_a = item[0].index(x)
                        indice_b = item[0].index(ps[ps_count+4]) 
                        type = item[0][indice_a+1:indice_b]
                        #print("Tipo: ",type)
                        ps_found = True
                        break
                    else:
                        #print("False")
                        ps_count += 1

                if ps_found:
                    #print("Se procede con designar el tipo.")
                    #print("Valor a insertar: "+str(item[1]))
                    json_inner = {}
                    json_inner['type'] = str(type)
                    json_inner['value'] = str(item[1])

                    name = item[0][0:indice_a]
                    json_obj[name] = json_inner
                    
                else:
                    #print("Valor a insertar: "+str(item[1]))
                    json_obj[item[0]] = str(item[1])
                    
            #Termina con el json pra reiniciar de nuevo if ps_found:
            data.append(json_obj)
            print(data)

        with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
            jsonf.write(json.dumps(data, indent=4, ensure_ascii=False, separators=(',', ':')))

    return data #Para poder ser insertada en la base de datos postgres correctamente.
 
load_csv("model/entities.csv")

Importar un documento csv y convertirlo en JSON
[{'entityID': 'ENT-WO-001', 'entityType': 'WeatherObserved', 'municipality': {'type': 'Text', 'value': 'NA'}, 'TimeInstant': {'type': 'DateTime', 'value': '2021-04-07T09:39:20.516Z'}, 'address': {'type': 'Text', 'value': 'Calle de la Cruz,28100, Alcobendas, Madrid, España'}, 'dataProvider': {'type': 'Text', 'value': 'https://dataprovider.com'}, 'location': {'type': 'geo:json', 'value': '{"coordinates":[-3.6318964,40.5400098],"type":"Point"}'}, 'name': {'type': 'Text', 'value': 'Estación Meteo 1'}, 'battery': {'type': 'Number', 'value': '10.0'}, 'source': {'type': 'Text', 'value': 'https://source.com'}, 'UV': {'type': 'Number', 'value': '6.41'}, 'absoluteHumidity': {'type': 'Number', 'value': '404.0'}, 'atmosphericPressure': {'type': 'Number', 'value': '770.27'}, 'corrosion': {'type': 'Number', 'value': '50.0'}, 'corrosionIndex': {'type': 'Number', 'value': '0.5'}, 'dailyEvapotranspiration': {'type': 'Number', 'value': '2.97'}, 'dailyPreci

[{'entityID': 'ENT-WO-001',
  'entityType': 'WeatherObserved',
  'municipality': {'type': 'Text', 'value': 'NA'},
  'TimeInstant': {'type': 'DateTime', 'value': '2021-04-07T09:39:20.516Z'},
  'address': {'type': 'Text',
   'value': 'Calle de la Cruz,28100, Alcobendas, Madrid, España'},
  'dataProvider': {'type': 'Text', 'value': 'https://dataprovider.com'},
  'location': {'type': 'geo:json',
   'value': '{"coordinates":[-3.6318964,40.5400098],"type":"Point"}'},
  'name': {'type': 'Text', 'value': 'Estación Meteo 1'},
  'battery': {'type': 'Number', 'value': '10.0'},
  'source': {'type': 'Text', 'value': 'https://source.com'},
  'UV': {'type': 'Number', 'value': '6.41'},
  'absoluteHumidity': {'type': 'Number', 'value': '404.0'},
  'atmosphericPressure': {'type': 'Number', 'value': '770.27'},
  'corrosion': {'type': 'Number', 'value': '50.0'},
  'corrosionIndex': {'type': 'Number', 'value': '0.5'},
  'dailyEvapotranspiration': {'type': 'Number', 'value': '2.97'},
  'dailyPrecipitation':

In [519]:
import psycopg2

"""

Si se imprime la consulta completa, algunos campos salen como None puesto que desde el CSV no
se carga el valor indicado para cada una de las columnas de la tabla, al imprimir solo
entityID y timeinstant si se obtiene el resultado esperado.

La lectura de las columnas a modificar así como de los datos a insertar se lee guiandose
de la data que provee el JSON, de esta manera se pueden ahorrar muchas lineas de código
al evitar escribir una extensa consulta estatica.

"""

# Puedes cambiar los datos de conexión si tu base de datos está en un PC distinto
db_host  = "localhost"
db_port  = "5432"
username = "warpcom"
password = "p4ssword"
database = "warpcom"


def insert_all_rows():
    """Inserta todas las filas del CSV model/entities.csv en la base de datos"""
    with psycopg2.connect(f"host={db_host} port={db_port} dbname={database} user={username} password={password}") as conn:
        # Limpiamos la tabla weatherobserved para que no haya problemas de claves duplicadas
        with conn.cursor() as cur:
            cur.execute("DELETE FROM environment_weatherobserved")
            conn.commit()
        # Cargamos los datos
        print("Se va a cargar la data.")
        for entity in load_csv("model/entities.csv"):
            
            #print("Entidad: "+str(entity))
            insert_row(conn, entity)
            #break --> solo debug
            
        # Y nos aseguramos de que se han cargado
        with conn.cursor() as cur:
            cur.execute("SELECT entityid, timeinstant from environment_weatherobserved")
            return cur.fetchall()

def insert_row(conn, entity):
    # TODO: implementa esta funcion
    #print("Se insertan n filas en la BD.")

    cols = ''
    #Definimos las columnas
    for item in entity:
        cols = cols + str(item) + ','
    cols = cols[:-1]
    #print("Columnas: "+str(cols))

    all_data = ''
    #Definimos los valores a insertar.
    for item in entity:
        value = entity.get(item)
        #print("Valor extraido: "+str(item)+" --> "+str(value))

        try:
            if value.get("type"):
                #print(value.get("type"))
                #Segun el tipo de dato encontrado, se establece como se va a insertar
                #un dato en particular en la BD, util segun lo requiera el Cliente.
                if value.get("type") == "Text":
                    #print("Texto encontrado!")
                    all_data = all_data + "'"+str(value.get("value")) + "',"

                elif value.get("type") == "DateTime":
                    #print("DateTime encontrado!")
                    all_data = all_data + "'"+str(value.get("value")) + "',"

                elif value.get("type") == "geo:json":
                    #print("geo:json encontrado!")
                    all_data = all_data + "ST_GeomFromGeoJSON('"+str(value.get("value")) + "'),"

                elif value.get("type") == "Number":
                    #print("Number encontrado!")
                    all_data = all_data + ""+str(value.get("value")) + ","

        except:
            #print("Cadena encontrada")
            all_data = all_data + "'"+str(value) + "',"

    all_data = all_data[:-1]

    print("All data: "+str(all_data))

    query = """INSERT INTO environment_weatherobserved ("""+str(cols)+""") VALUES ("""+str(all_data)+""")"""

    with conn.cursor() as cur:
        cur.execute(query)

insert_all_rows()

Se va a cargar la data.
Importar un documento csv y convertirlo en JSON
[{'entityID': 'ENT-WO-001', 'entityType': 'WeatherObserved', 'municipality': {'type': 'Text', 'value': 'NA'}, 'TimeInstant': {'type': 'DateTime', 'value': '2021-04-07T09:39:20.516Z'}, 'address': {'type': 'Text', 'value': 'Calle de la Cruz,28100, Alcobendas, Madrid, España'}, 'dataProvider': {'type': 'Text', 'value': 'https://dataprovider.com'}, 'location': {'type': 'geo:json', 'value': '{"coordinates":[-3.6318964,40.5400098],"type":"Point"}'}, 'name': {'type': 'Text', 'value': 'Estación Meteo 1'}, 'battery': {'type': 'Number', 'value': '10.0'}, 'source': {'type': 'Text', 'value': 'https://source.com'}, 'UV': {'type': 'Number', 'value': '6.41'}, 'absoluteHumidity': {'type': 'Number', 'value': '404.0'}, 'atmosphericPressure': {'type': 'Number', 'value': '770.27'}, 'corrosion': {'type': 'Number', 'value': '50.0'}, 'corrosionIndex': {'type': 'Number', 'value': '0.5'}, 'dailyEvapotranspiration': {'type': 'Number', 'valu

[('ENT-WO-001',
  datetime.datetime(2021, 4, 7, 2, 39, 20, 516000, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=61200)))),
 ('ENT-WO-002',
  datetime.datetime(2021, 4, 7, 2, 39, 20, 897000, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=61200)))),
 ('ENT-WO-003',
  datetime.datetime(2021, 4, 7, 2, 39, 21, 282000, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=61200)))),
 ('ENT-WO-004',
  datetime.datetime(2021, 4, 7, 2, 39, 21, 670000, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=61200)))),
 ('ENT-WO-005',
  datetime.datetime(2021, 4, 7, 2, 39, 22, 80000, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=61200)))),
 ('ENT-WO-002',
  datetime.datetime(2021, 4, 7, 4, 44, 12, 897000, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=61200)))),
 ('ENT-WO-002',
  datetime.datetime(2021, 4, 7, 11, 31, 1, 897000, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=61200)))),
 ('ENT-WO-002',
  datetime.d

In [520]:
import os
os.environ["PGPASSWORD"] = "p4ssword"

#Se usa UTC para evitar que la hora del timestamp sea modificado y genere un resultado errado.
#Se usa un CAST para convertir el timestamp a DATE y poder así encontrar coincidencias.
#El promedio de la temperatura se obtiene con la función AVG, se agrega el CAST a Float para garantizar resultados en decimal.
#Como el entityid es uno solo por la condición where puede usarse en conjunto con el valor de dia para obtener el resultado.
#Se ordena de forma ascendente por la columna tipo DATE.

# Cambia los datos de conexion (-h, -p) si tu servidor postgres está en otro equipo
!psql -h localhost -p 5432 -U warpcom -d warpcom -c "SELECT entityid, CAST(timeinstant at time zone 'UTC' as date) AS dia, AVG(CAST(temperature as float)) AS temperatura_media FROM environment_weatherobserved WHERE entityid = 'ENT-WO-002' GROUP BY dia, entityid ORDER BY dia"

  entityid  |    dia     | temperatura_media 
------------+------------+-------------------
 ENT-WO-002 | 2021-04-07 |                28
 ENT-WO-002 | 2021-04-08 |  24.3333333333333
 ENT-WO-002 | 2021-04-09 |  14.3333333333333
(3 rows)



In [521]:
import os
os.environ["PGPASSWORD"] = "p4ssword"

# Cambia los datos de conexion (-h, -p) si tu servidor postgres está en otro equipo
!psql -h localhost -p 5432 -U warpcom -d warpcom -c "SELECT entityid, timeinstant at time zone 'UTC-2', temperature FROM environment_weatherobserved P1 WHERE timeinstant = (SELECT MAX(timeinstant) FROM environment_weatherobserved P2 WHERE P2.entityid = P1.entityid) ORDER BY entityid"

  entityid  |        timezone         | temperature 
------------+-------------------------+-------------
 ENT-WO-001 | 2021-04-07 11:39:20.516 |          14
 ENT-WO-002 | 2021-04-09 20:53:35.897 |          15
 ENT-WO-003 | 2021-04-11 16:21:42.282 |          12
 ENT-WO-004 | 2021-04-10 18:33:32.67  |          21
 ENT-WO-005 | 2021-04-12 19:44:11.08  |          28
(5 rows)

